In [ ]:
import os
import re
import time
import requests
import shutil
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.alert import Alert
import chromedriver_autoinstaller

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
chromedriver_autoinstaller.install()
browser = webdriver.Chrome(options=options)

url = "https://www.ksa.or.kr/ksi/4982/subview.do"
headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36",
    "Accept-Language":"ko-KR,ko"
    }
browser.get(url)

index = 1
report_name = []
report_year = []
report_sec = []
report_standard_icon = []
report_url = []
standard_list = {
    "/Web-home/fnct/krca/JW_Krca_basic/images/ico_use1.gif" : "GRI",
    "/Web-home/fnct/krca/JW_Krca_basic/images/ico_use2.gif" : "AA1000",
    "/Web-home/fnct/krca/JW_Krca_basic/images/ico_use3.gif" : "UNGC",
    "/Web-home/fnct/krca/JW_Krca_basic/images/ico_use4.gif" : "BSR",
    "/Web-home/fnct/krca/JW_Krca_basic/images/ico_use7.gif" : "ISO 26000",
    "/Web-home/fnct/krca/JW_Krca_basic/images/ico_use8.gif" : "IR"
}

res_max = requests.get(browser.current_url, headers=headers)
res_max.raise_for_status()
soup_max = BeautifulSoup(res_max.text, "lxml")
max = soup_max.find("div", attrs={"class":"_inner"}).find("a", attrs={"class":"_last"})["href"][22:-2]

while index <= int(max) + 1:
    try :
        browser.execute_script("javascript:page_link('{}')".format(index))
        res = requests.get(browser.current_url, headers=headers)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "lxml")
        list = soup.find("tbody").find_all("tr")
        for report in list :
            report_name.append(report.find("td").get_text())
            year = re.sub(r'[^0-9]', '', report.find("td").find_next_sibling("td").find_next_sibling("td").get_text())
            report_year.append(year)
            report_sec.append(report.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").get_text())
            report_standard_icon.append(report.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td"))
            report_download = report.find("a")["href"]
            report_num = int(report_download[12:-12])
            report_url.append("https://www.ksa.or.kr" + report_download)
        index += 1
    except Exception as e:
        print(e)
        browser.refresh()
        time.sleep(30)

report_standard = []
for report in report_standard_icon :
    standard = []
    for index, stand in standard_list.items() :
        if index in str(report) :
            standard.append(stand)
    report_standard.append(", ".join(standard))


d = {'year' : report_year, 'name' : report_name, 'sector' : report_sec, 'standard' : report_standard, 'url' : report_url}
df = pd.DataFrame(d)
print(df)
# df.to_excel('./Report_crawler/Report_list.xlsx', sheet_name = "report_list")
